In [1]:
import pandas as pd
import numpy as np

label = pd.read_excel('./data/label.xlsx')
data = pd.read_csv("./data/품질전처리후데이터.csv")


In [2]:
# --- 1. 'TAG_MIN'을 datetime으로 변환 (오류 방지) ---
try:
    if not pd.api.types.is_datetime64_any_dtype(data['TAG_MIN']):
        data['TAG_MIN'] = pd.to_datetime(data['TAG_MIN'])
        print("'TAG_MIN' datetime으로 변환 완료.")
except Exception as e:
    print(f"datetime 변환 오류: {e}")

# --- 2. Raw 데이터(data) 결측치 제거 ---
original_rows = len(data)
data.dropna(inplace=True)
print(f"'data' 결측치 제거: {original_rows} -> {len(data)} 행")

# --- 3. Label에서 병합할 정보 추출 ---
# (불량 유형, 작업자, 품번 등 필요한 모든 정보)
defect_cols = [
    '원소재', '변형(폭)', '내폭', '외폭', '형상파손', 
    '경도', '인장경도', '초도품', 'TEST', '경도측정'
]
base_cols = ['배정번호', '양품수량', '불량수량', '작업자', '품번']

# '배정번호' 중복이 없는지 확인하고 병합할 테이블 생성
if label['배정번호'].duplicated().any():
    print("경고: 'label' 데이터에 '배정번호' 중복이 있습니다. 첫 번째 값만 사용합니다.")
    label_to_merge = label[base_cols + defect_cols].drop_duplicates(subset=['배정번호'], keep='first')
else:
    label_to_merge = label[base_cols + defect_cols]
    print("'label' 데이터 '배정번호' 중복 없음. 병합 준비 완료.")

# --- 4. 'data' (좌) 기준으로 'label_to_merge' (우) 병합 ---
print("Left Merge 시작...")
df_merged_full = pd.merge(
    data, 
    label_to_merge, 
    on='배정번호', 
    how='left' # data(왼쪽)의 모든 행을 유지
)
print("Left Merge 완료.")

# --- 5. 최종 '불량률' 및 '내폭_불량률' 컬럼 생성 ---
df_merged_full['총생산수량'] = df_merged_full['양품수량'] + df_merged_full['불량수량']
df_merged_full['총생산수량'] = df_merged_full['총생산수량'].replace(0, 1) # 0으로 나누기 방지

df_merged_full['불량률'] = (df_merged_full['불량수량'] / df_merged_full['총생산수량']) * 100
df_merged_full['내폭_불량률'] = (df_merged_full['내폭'] / df_merged_full['총생산수량']) * 100

print(f"통합 데이터프레임 'df_merged_full' 생성 완료: {df_merged_full.shape}")
print("\n--- df_merged_full.head() ---")
print(df_merged_full.head())

# --- 1. df_merged_full (290만 행)의 배정번호 확인 ---
print("--- [1] df_merged_full (290만 행)의 배정번호 ---")

# '배정번호'의 고유한 값들을 배열(array)로 추출
unique_batches_full = df_merged_full['배정번호'].unique()

print(f"고유한 배정번호 개수: {len(unique_batches_full)}")
print(f"고유한 배정번호 목록 (상위 10개 샘플): {unique_batches_full[:10]}")


# --- 2. label (136행)의 배정번호 확인 ---
print("\n--- [2] label (136행)의 배정번호 ---")

# '배정번호'의 고유한 값들을 배열(array)로 추출
unique_batches_label = label['배정번호'].unique()

print(f"고유한 배정번호 개수: {len(unique_batches_label)}")
print(f"고유한 배정번호 목록 (상위 10개 샘플): {unique_batches_label[:10]}")


# --- 3. 두 목록이 일치하는지 확인 ---
# (두 목록을 정렬해서 비교)
is_identical = (np.sort(unique_batches_full) == np.sort(unique_batches_label)).all()
print(f"\n--- [3] 두 목록 일치 여부 ---")
print(f"두 데이터셋의 배정번호 목록이 일치합니까? {is_identical}")

'TAG_MIN' datetime으로 변환 완료.
'data' 결측치 제거: 2939722 -> 2935045 행
'label' 데이터 '배정번호' 중복 없음. 병합 준비 완료.
Left Merge 시작...
Left Merge 완료.
통합 데이터프레임 'df_merged_full' 생성 완료: (2935045, 38)

--- df_merged_full.head() ---
   Unnamed: 0             TAG_MIN    배정번호  건조 1존 OP  건조 2존 OP  건조로 온도 1 Zone  \
0           2 2022-01-03 11:22:09  102410   75.6776   32.1592        98.8533   
1           3 2022-01-03 11:22:11  102410   75.8656   30.8312        98.7918   
2           4 2022-01-03 11:22:12  102410   73.6468   29.5274        98.7918   
3           5 2022-01-03 11:22:13  102410   76.0051   29.5927        98.7918   
4           6 2022-01-03 11:22:14  102410   75.9804   29.5291        98.7918   

   건조로 온도 2 Zone      세정기   소입1존 OP  소입2존 OP  ...  외폭  형상파손  경도  인장경도  초도품  \
0       99.14600  68.4386  78.10990  61.5414  ...   0     0   0     0    0   
1       99.17675  68.4999  77.50725  60.6663  ...   0     0   0     0    0   
2       99.20750  68.4386  76.02620  61.1634  ...   0     0   0     0    0

In [3]:
# --- '배정번호' 102410 데이터 분리 ---

# df_merged_full에서 '배정번호' 컬럼의 값이 102410인 모든 행을 선택합니다.
df_102410 = df_merged_full[df_merged_full['배정번호'] == 102410].copy()

# --- 결과 확인 ---
print(f"--- 'df_102410' 생성 완료 ---")
print(f"총 데이터 개수: {len(df_102410)} 행")
print("\n'df_102410' 상위 5개 샘플:")
print(df_102410.head())

--- 'df_102410' 생성 완료 ---
총 데이터 개수: 7340 행

'df_102410' 상위 5개 샘플:
   Unnamed: 0             TAG_MIN    배정번호  건조 1존 OP  건조 2존 OP  건조로 온도 1 Zone  \
0           2 2022-01-03 11:22:09  102410   75.6776   32.1592        98.8533   
1           3 2022-01-03 11:22:11  102410   75.8656   30.8312        98.7918   
2           4 2022-01-03 11:22:12  102410   73.6468   29.5274        98.7918   
3           5 2022-01-03 11:22:13  102410   76.0051   29.5927        98.7918   
4           6 2022-01-03 11:22:14  102410   75.9804   29.5291        98.7918   

   건조로 온도 2 Zone      세정기   소입1존 OP  소입2존 OP  ...  외폭  형상파손  경도  인장경도  초도품  \
0       99.14600  68.4386  78.10990  61.5414  ...   0     0   0     0    0   
1       99.17675  68.4999  77.50725  60.6663  ...   0     0   0     0    0   
2       99.20750  68.4386  76.02620  61.1634  ...   0     0   0     0    0   
3       99.14600  68.4386  75.88260  61.1124  ...   0     0   0     0    0   
4       99.20750  68.4386  75.75040  59.5448  ...   0     0   0

TAG MIN 결측치

In [4]:
# TAG_MIN을 datetime 형식으로 변환 (초 단위 정렬 보장)
df_102410["TAG_MIN"] = pd.to_datetime(df_102410["TAG_MIN"])
df_102410 = df_102410.sort_values("TAG_MIN").reset_index(drop=True)

# 연속된 초 구간 전체 생성 (최소~최대)
full_range = pd.date_range(
    start=df_102410["TAG_MIN"].min(),
    end=df_102410["TAG_MIN"].max(),
    freq="s"   # 초 단위 (Seconds)
)

# 누락된 시간대 찾기
missing_times = full_range.difference(df_102410["TAG_MIN"])

# 결과 출력
if len(missing_times) == 0:
    print("결측 초 없음")
else:
    print(f"결측 초 존재: 총 {len(missing_times)}개 누락")
    print("누락된 시간대 예시 (앞 20개):")
    print(missing_times)

결측 초 존재: 총 155개 누락
누락된 시간대 예시 (앞 20개):
DatetimeIndex(['2022-01-03 11:22:10', '2022-01-03 11:22:28',
               '2022-01-03 11:30:46', '2022-01-03 11:31:08',
               '2022-01-03 11:31:12', '2022-01-03 11:31:22',
               '2022-01-03 11:35:09', '2022-01-03 11:35:12',
               '2022-01-03 11:35:14', '2022-01-03 11:35:16',
               ...
               '2022-01-03 13:10:44', '2022-01-03 13:10:46',
               '2022-01-03 13:15:33', '2022-01-03 13:15:41',
               '2022-01-03 13:19:56', '2022-01-03 13:20:01',
               '2022-01-03 13:20:03', '2022-01-03 13:24:01',
               '2022-01-03 13:24:24', '2022-01-03 13:24:35'],
              dtype='datetime64[ns]', length=155, freq=None)


In [5]:
# TAG_MIN을 datetime으로 변환 후 정렬
df_102410["TAG_MIN"] = pd.to_datetime(df_102410["TAG_MIN"])
df_102410 = df_102410.sort_values("TAG_MIN").reset_index(drop=True)

# object형 불필요 컬럼 제거
drop_cols = ["작업자", "품번"]
existing_drop = [c for c in drop_cols if c in df_102410.columns]
df_102410 = df_102410.drop(columns=existing_drop)
print(f"불필요 문자열 컬럼 제거 완료: {existing_drop}")

# TAG_MIN을 인덱스로 설정
df_102410 = df_102410.set_index("TAG_MIN")

# 초 단위 연속 구간 생성
full_range = pd.date_range(
    start=df_102410.index.min(),
    end=df_102410.index.max(),
    freq="s"
)

# reindex → 결측 시간 삽입
df_full = df_102410.reindex(full_range)

# 선형 보간 (시간 기준)
df_full = df_full.interpolate(method="linear", limit_direction="both")
print("선형 보간(linear interpolation) 완료")

# 숫자형 컬럼 평균으로 남은 결측치 처리
numeric_cols = df_full.select_dtypes(include=[np.number]).columns
if df_full[numeric_cols].isnull().values.any():
    print("WARNING: 일부 남은 결측치는 평균으로 보완합니다.")
    df_full[numeric_cols] = df_full[numeric_cols].fillna(df_full[numeric_cols].mean(numeric_only=True))

# 인덱스를 'TAG_MIN' 컬럼으로 복원 (Flask 호환)
df_full = df_full.reset_index().rename(columns={'index': 'TAG_MIN'})
df_full["TAG_MIN"] = pd.to_datetime(df_full["TAG_MIN"])  # 혹시 문자열로 남아있을 경우 대비

print("'TAG_MIN' 컬럼 복원 완료")
print(f"총 {len(df_full) - len(df_102410)}개의 초가 추가되어, 총 {len(df_full)}개 시각으로 확장되었습니다.")
print(df_full.head(5))

불필요 문자열 컬럼 제거 완료: ['작업자', '품번']
선형 보간(linear interpolation) 완료
'TAG_MIN' 컬럼 복원 완료
총 155개의 초가 추가되어, 총 7495개 시각으로 확장되었습니다.
              TAG_MIN  Unnamed: 0      배정번호  건조 1존 OP  건조 2존 OP  \
0 2022-01-03 11:22:09         2.0  102410.0   75.6776   32.1592   
1 2022-01-03 11:22:10         2.5  102410.0   75.7716   31.4952   
2 2022-01-03 11:22:11         3.0  102410.0   75.8656   30.8312   
3 2022-01-03 11:22:12         4.0  102410.0   73.6468   29.5274   
4 2022-01-03 11:22:13         5.0  102410.0   76.0051   29.5927   

   건조로 온도 1 Zone  건조로 온도 2 Zone       세정기    소입1존 OP   소입2존 OP  ...   외폭  \
0       98.85330      99.146000  68.43860  78.109900  61.54140  ...  0.0   
1       98.82255      99.161375  68.46925  77.808575  61.10385  ...  0.0   
2       98.79180      99.176750  68.49990  77.507250  60.66630  ...  0.0   
3       98.79180      99.207500  68.43860  76.026200  61.16340  ...  0.0   
4       98.79180      99.146000  68.43860  75.882600  61.11240  ...  0.0   

   형상파손   경도  인장경도 

In [6]:
df_full

,TAG_MIN,Unnamed: 0,배정번호,건조 1존 OP,건조 2존 OP,건조로 온도 1 Zone,건조로 온도 2 Zone,세정기,소입1존 OP,소입2존 OP,...,외폭,형상파손,경도,인장경도,초도품,TEST,경도측정,총생산수량,불량률,내폭_불량률
0,2022-01-03 11:22:09,2.0,102410.0,75.6776,32.1592,98.85330,99.146000,68.43860,78.109900,61.54140,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15163.0,0.019785,0.019785
1,2022-01-03 11:22:10,2.5,102410.0,75.7716,31.4952,98.82255,99.161375,68.46925,77.808575,61.10385,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15163.0,0.019785,0.019785
2,2022-01-03 11:22:11,3.0,102410.0,75.8656,30.8312,98.79180,99.176750,68.49990,77.507250,60.66630,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15163.0,0.019785,0.019785
3,2022-01-03 11:22:12,4.0,102410.0,73.6468,29.5274,98.79180,99.207500,68.43860,76.026200,61.16340,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15163.0,0.019785,0.019785
4,2022-01-03 11:22:13,5.0,102410.0,76.0051,29.5927,98.79180,99.146000,68.43860,75.882600,61.11240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15163.0,0.019785,0.019785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7490,2022-01-03 13:26:59,7337.0,102410.0,75.8076,25.2268,99.45610,98.569300,70.15330,62.631700,56.27940,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15163.0,0.019785,0.019785
7491,2022-01-03 13:27:00,7338.0,102410.0,73.7405,27.6011,99.46540,98.638200,70.28700,64.931300,54.53070,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15163.0,0.019785,0.019785
7492,2022-01-03 13:27:01,7339.0,102410.0,75.9631,27.4567,99.46540,98.576600,70.16440,68.220200,57.74380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15163.0,0.019785,0.019785
7493,2022-01-03 13:27:02,7340.0,102410.0,76.1320,27.4614,99.52690,98.576600,70.22570,66.559000,54.70290,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15163.0,0.019785,0.019785


## onnx 모델 적용

## 웹페이지

In [7]:
# TAG_MIN 컬럼이 datetime 형식인지 확인
if df_full["TAG_MIN"].dtype == "object":
    df_full["TAG_MIN"] = pd.to_datetime(df_full["TAG_MIN"])
elif not pd.api.types.is_datetime64_any_dtype(df_full["TAG_MIN"]):
    df_full["TAG_MIN"] = pd.to_datetime(df_full["TAG_MIN"])

# 정렬 및 인덱스 리셋
df_full = df_full.sort_values("TAG_MIN").reset_index(drop=True)

# 시작 시각 기준으로 300초 이후 시점 계산
start_time = df_full["TAG_MIN"].iloc[0]
split_time = start_time + pd.Timedelta(seconds=300)

#  처음 300초 구간 (warm-up)
df_full_before = df_full[df_full["TAG_MIN"] < split_time].reset_index(drop=True)

# 이후 구간만 남기기 (Flask용)
df_full = df_full[df_full["TAG_MIN"] > split_time].reset_index(drop=True)

print(f"warm-up 데이터: {len(df_full_before)} rows ({df_full_before['TAG_MIN'].iloc[0]} ~ {df_full_before['TAG_MIN'].iloc[-1]})")
print(f"Flask 실시간 데이터: {len(df_full)} rows ({df_full['TAG_MIN'].iloc[0]} ~ {df_full['TAG_MIN'].iloc[-1]})")


warm-up 데이터: 300 rows (2022-01-03 11:22:09 ~ 2022-01-03 11:27:08)
Flask 실시간 데이터: 7194 rows (2022-01-03 11:27:10 ~ 2022-01-03 13:27:03)


## 최종 모니터링 시스템
- 접속 정보 : [127.0.0.1:5432](127.0.0.1:5432)

In [8]:
import os, time, pickle
import numpy as np
import pandas as pd
from flask import Flask, render_template, jsonify
from threading import Lock
import onnxruntime as ort

# =========================================================
#  StandardScaler 로드
# =========================================================
with open('./scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# 컬럼명 -> scaler 인덱스 매핑
COLUMN_TO_SCALER_IDX = {
    "건조로 온도 2 Zone": 3,
    "소입로 온도 1 Zone": 10,
    "소입로 온도 4 Zone": 13,
}

print(f"StandardScaler loaded: {len(scaler.mean_)} features")

# =========================================================
#  데이터 준비
# =========================================================
# df_full이 이미 존재한다고 가정
df_full["TAG_MIN"] = pd.to_datetime(df_full["TAG_MIN"])
df_full = df_full.sort_values("TAG_MIN").reset_index(drop=True)

start_time = df_full["TAG_MIN"].iloc[0]
split_time = start_time + pd.Timedelta(seconds=300)

# 초기 300초 데이터 (warm-up)
df_full_before = df_full[df_full["TAG_MIN"] < split_time].reset_index(drop=True)
# 실시간 시뮬레이션용 데이터 (300초 이후)
df_full_after = df_full[df_full["TAG_MIN"] >= split_time].reset_index(drop=True)

print(f"warm-up 데이터: {len(df_full_before)} rows ({df_full_before['TAG_MIN'].iloc[0]} ~ {df_full_before['TAG_MIN'].iloc[-1]})")
print(f"Flask 실시간 데이터: {len(df_full_after)} rows ({df_full_after['TAG_MIN'].iloc[0]} ~ {df_full_after['TAG_MIN'].iloc[-1]})")

# =========================================================
#  전역 상태 변수
# =========================================================
df = df_full_after.copy()
current_index = 0
last_update_time = time.time()
index_lock = Lock()

# 전체 데이터 (예측용 슬라이딩 윈도우를 위해)
df_combined = pd.concat([df_full_before, df_full_after], ignore_index=True)
df_combined = df_combined.sort_values("TAG_MIN").reset_index(drop=True)

sections = {
    "dry_op": ["건조 1존 OP", "건조 2존 OP"],
    "dry_temp": ["건조로 온도 1 Zone", "건조로 온도 2 Zone"],
    "wash": ["세정기"],
    "quench": ["소입1존 OP", "소입2존 OP", "소입3존 OP", "소입4존 OP"],
    "furnace_op": ["소입로 CP 값"],
    "furnace_temp": ["소입로 온도 1 Zone", "소입로 온도 2 Zone", "소입로 온도 3 Zone", "소입로 온도 4 Zone"]
}

# =========================================================
#  예측 관련 설정 (ONNX 모델)
# =========================================================
FORECAST_COLS = [
    ("건조로 온도 2 Zone", 98.88),
    ("소입로 온도 1 Zone", 850.13),
    ("소입로 온도 4 Zone", 858.71),
]
N_IN, N_OUT = 30, 60  # 입력 30스텝(10초 단위), 출력 60스텝(10초 단위)

model_paths = {
    "건조로 온도 2 Zone": "./onnx/LSTM_건조로 온도 2 Zone_0.01_L1Loss.onnx",
    "소입로 온도 1 Zone": "./onnx/LSTM_소입로 온도 1 Zone_0.01_L1Loss.onnx",
    "소입로 온도 4 Zone": "./onnx/LSTM_소입로 온도 4 Zone_0.01_L1Loss.onnx",
}

print("\n모델 로딩 시작...")
onnx_sessions = {}
for name, path in model_paths.items():
    try:
        sess = ort.InferenceSession(path, providers=["CPUExecutionProvider"])
        onnx_sessions[name] = sess
        print(f"  - {name}: 로드 성공")
    except Exception as e:
        print(f"  - {name}: 로드 실패 - {e}")

forecast_cache = {
    "labels": [],
    "series": {"dry_temp_2": [], "quench_temp_1": [], "quench_temp_4": []},
    "alarms": {"dry_temp_2": False, "quench_temp_1": False, "quench_temp_4": False},
    "last_from": None
}

def _series_key(colname: str) -> str:
    return {
        "건조로 온도 2 Zone": "dry_temp_2",
        "소입로 온도 1 Zone": "quench_temp_1",
        "소입로 온도 4 Zone": "quench_temp_4"
    }.get(colname, colname)

def _build_forecast_times(start_ts: pd.Timestamp) -> list[str]:
    times = pd.date_range(start_ts + pd.Timedelta(seconds=10), periods=N_OUT, freq="10s")
    return [t.strftime("%Y-%m-%d %H:%M:%S") for t in times]

def _predict_next_5min(last300_df: pd.DataFrame) -> dict:
    """
    최근 300초 데이터를 기반으로 향후 5분(60스텝) 온도 예측
    - 입력: 정규화된 데이터 (30스텝)
    - 출력: 역정규화된 원본 스케일 예측값 (60스텝)
    """
    print(f"\n예측 함수 호출 - 입력 데이터 크기: {len(last300_df)}")
    
    results, alarms = {}, {}
    
    # 10초 단위로 리샘플링
    resampled = last300_df.set_index("TAG_MIN").resample("10s").mean().interpolate()
    print(f"리샘플링 후 데이터 크기: {len(resampled)}")
    
    last_ts = resampled.index[-1]
    labels = _build_forecast_times(last_ts)
    print(f"예측 시간 라벨 생성 완료: {len(labels)}개")

    for colname, thr in FORECAST_COLS:
        print(f"\n[{colname}] 예측 시작")
        
        if colname not in onnx_sessions:
            print(f"  경고: {colname} 모델이 로드되지 않음")
            continue
            
        sess = onnx_sessions[colname]
        input_name = sess.get_inputs()[0].name
        output_name = sess.get_outputs()[0].name

        # STEP 1: 원본 데이터 추출 (최근 30스텝)
        if colname not in resampled.columns:
            print(f"  오류: {colname} 컬럼이 데이터에 없음")
            print(f"  사용 가능한 컬럼: {resampled.columns.tolist()}")
            continue
            
        seq_original = resampled[colname].values[-N_IN:].astype(np.float32)
        print(f"  원본 데이터 추출: {len(seq_original)}개, 범위: {seq_original.min():.2f}~{seq_original.max():.2f}")

        # STEP 2: 입력 데이터 정규화 (float32로 명시적 변환)
        scaler_idx = COLUMN_TO_SCALER_IDX[colname]
        mean_val = np.float32(scaler.mean_[scaler_idx])
        scale_val = np.float32(scaler.scale_[scaler_idx])
        
        # 정규화 후 명시적으로 float32 변환
        seq_normalized = ((seq_original - mean_val) / scale_val).astype(np.float32)
        print(f"  정규화 완료: mean={mean_val:.2f}, scale={scale_val:.2f}, dtype={seq_normalized.dtype}")

        # STEP 3: ONNX 모델 추론
        X_in = seq_normalized.reshape(1, N_IN, 1)
        print(f"  입력 shape: {X_in.shape}, dtype: {X_in.dtype}")
        
        y_pred_normalized = sess.run([output_name], {input_name: X_in})[0].flatten()
        print(f"  모델 추론 완료: {len(y_pred_normalized)}개 예측값")

        # STEP 4: 예측값 역정규화
        y_pred_original = y_pred_normalized * scale_val + mean_val
        print(f"  역정규화 완료: 범위 {y_pred_original.min():.2f}~{y_pred_original.max():.2f}")

        results[_series_key(colname)] = y_pred_original.tolist()

        # STEP 5: 임계값 비교
        if colname == "소입로 온도 4 Zone":
            alarm = any(v > thr for v in y_pred_original)
        else:
            alarm = any(v < thr for v in y_pred_original)
        alarms[_series_key(colname)] = alarm
        print(f"  알람 상태: {alarm} (임계값: {thr})")

    result_dict = {
        "labels": labels,
        "series": results,
        "alarms": alarms,
        "last_from": last_ts.strftime("%Y-%m-%d %H:%M:%S"),
    }
    
    print(f"\n예측 완료:")
    print(f"  - 라벨 수: {len(result_dict['labels'])}")
    print(f"  - 시리즈 키: {list(result_dict['series'].keys())}")
    for k, v in result_dict['series'].items():
        print(f"    - {k}: {len(v)}개 값")
    
    return result_dict

# =========================================================
#  초기 warm-up 예측
# =========================================================
print("\n초기 warm-up 예측 시작...")
try:
    if len(df_full_before) >= 300:
        init_win = df_full_before.tail(300).copy()
        print(f"warm-up 데이터 준비 완료: {len(init_win)}행")
        
        forecast_result = _predict_next_5min(init_win)
        forecast_cache.update(forecast_result)
        
        print(f"\n초기 예측 완료 - 기준 시각: {forecast_cache['last_from']}")
        print(f"forecast_cache 상태:")
        print(f"  - labels: {len(forecast_cache['labels'])}개")
        print(f"  - series keys: {list(forecast_cache['series'].keys())}")
        for k, v in forecast_cache['series'].items():
            print(f"    - {k}: {len(v)}개")
    else:
        print(f"경고: warm-up 데이터 부족 ({len(df_full_before)} < 300)")
except Exception as e:
    print(f"초기 예측 실패: {e}")
    import traceback
    traceback.print_exc()

# =========================================================
#  Flask 앱
# =========================================================
app = Flask(__name__)

@app.route("/")
def home():
    return render_template("main.html")

@app.route("/dashboard")
def dashboard():
    return render_template("dashboard.html")

@app.route("/latest")
def latest():
    global current_index, last_update_time, forecast_cache
    with index_lock:
        now = time.time()
        elapsed = now - last_update_time
        if elapsed >= 0.95:
            current_index = min(current_index + 1, len(df) - 1)
            last_update_time = now

        row = df.iloc[current_index]
        ts = str(row["TAG_MIN"])
        sec_values = {}
        for sec, cols in sections.items():
            vals = []
            for c in cols:
                v = float(row[c]) if c in df.columns else 0.0
                vals.append(v)
            sec_values[sec] = {"values": vals}

        # 1분(60초)마다 모델 재추론 - 슬라이딩 윈도우 방식
        if (current_index + 1) % 60 == 0:
            try:
                # df_combined에서 현재 시점 기준 최근 300초(300 rows) 추출
                combined_index = len(df_full_before) + current_index
                if combined_index >= 299:
                    recent_df = df_combined.iloc[combined_index - 299: combined_index + 1].copy()
                    forecast_result = _predict_next_5min(recent_df)
                    forecast_cache.update(forecast_result)
                    print(f"예측 갱신 완료 (index {current_index}): {forecast_cache['last_from']}")
            except Exception as e:
                print(f"예측 갱신 실패: {e}")
                import traceback
                traceback.print_exc()

        forecast_payload = {
            "labels": forecast_cache["labels"],
            "series": forecast_cache["series"],
            "alarms": forecast_cache["alarms"],
            "last_from": forecast_cache["last_from"],
            "thresholds": {
                "dry_temp_2": 98.88,
                "quench_temp_1": 850.13,
                "quench_temp_4": 858.71,
            }
        }

        return jsonify({
            "timestamp": ts,
            "sections": sec_values,
            "current_index": int(current_index),
            "total_rows": int(len(df)),
            "forecast": forecast_payload
        })

# =========================================================
#  HTML 템플릿 저장
# =========================================================
os.makedirs("templates", exist_ok=True)

# ---- main.html (예측 + 실시간 + 상세보기 버튼) ----
main_html = r"""<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>실시간 공정 불량률 예측 대시보드</title>
  <script src="https://cdn.jsdelivr.net/npm/chart.js@4.4.0/dist/chart.umd.min.js"></script>
  <style>
    body { font-family:'Malgun Gothic',Arial;margin:16px;background-color:#f5f7fa; }
    h1 { text-align:center;color:#2c3e50;margin:10px 0 6px;font-size:24px; }
    .btn { display:block;margin:8px auto 16px;padding:8px 18px;border:none;border-radius:6px;
           background:#007bff;color:#fff;font-size:14px;cursor:pointer; }
    .btn:hover { background:#0056b3; }
    .forecast-grid { display:grid;grid-template-columns:4fr 1fr;grid-row-gap:10px;grid-column-gap:14px;
                     margin:6px auto 18px;max-width:1280px; }
    .card { background:#fff;border-radius:8px;padding:10px;box-shadow:0 2px 4px rgba(0,0,0,0.06); }
    .card h3 { margin:4px 0 8px;font-size:14px;color:#34495e;text-align:left; }
    .traffic { display:flex;flex-direction:column;align-items:center;justify-content:center;
               height:100%;font-size:13px;color:#2c3e50; }
    .lamp { width:20px;height:20px;border-radius:50%;margin-bottom:6px;box-shadow:0 0 8px rgba(0,0,0,0.08);}
    .lamp.green{background:#2ecc71;} .lamp.red{background:#e74c3c;}
    canvas.forecast{width:100%!important;height:160px!important;}
    canvas.realtime{width:100%!important;height:180px!important;}
    .rt-grid{display:grid;grid-template-columns:1fr 1fr 1fr;gap:14px;max-width:1280px;margin:0 auto;}
    .status{text-align:center;font-size:12px;color:#555;margin-top:10px;}
  </style>
</head>
<body>
  <h1>실시간 공정 온도 예측 대시보드</h1>
  <button class="btn" onclick="window.location.href='/dashboard'">상세 공정 보기</button>
  <div class="forecast-grid">
    <div class="card"><h3>건조로 온도 2존 (하한 98.88)</h3><canvas id="fc_dry" class="forecast"></canvas></div>
    <div class="card traffic"><div class="lamp" id="lamp_dry"></div><div>건조 예측</div></div>
    <div class="card"><h3>소입로 온도 1존 (하한 850.13)</h3><canvas id="fc_quench" class="forecast"></canvas></div>
    <div class="card traffic"><div class="lamp" id="lamp_quench"></div><div>소입로 예측</div></div>
    <div class="card"><h3>소입로 온도 4존 (상한 858.71)</h3><canvas id="fc_furnace" class="forecast"></canvas></div>
    <div class="card traffic"><div class="lamp" id="lamp_furnace"></div><div>소입로 예측</div></div>
  </div>
  <div class="rt-grid">
    <div class="card"><h3>건조로 온도 2존 (실시간)</h3><canvas id="rt_dry" class="realtime"></canvas></div>
    <div class="card"><h3>소입로 온도 1존 (실시간)</h3><canvas id="rt_quench" class="realtime"></canvas></div>
    <div class="card"><h3>소입로 온도 4존 (실시간)</h3><canvas id="rt_furnace" class="realtime"></canvas></div>
  </div>
  <p id="status" class="status">데이터 수신 중...</p>
  <script>
    const fcCharts={},rtCharts={};
    const COLORS=["#3498db","#e67e22","#2ecc71"];
    const thresholds={dry:98.88,quench:850.13,furnace:858.71};

    function makeChart(ctx,label,thr,minY,maxY){
      const datasets=[{label:label,data:[],borderColor:COLORS[0],borderWidth:2,fill:false,pointRadius:1}];
      if(thr!==null){
        datasets.push({label:'임계값',data:[],borderColor:'#ff0000',borderWidth:1.5,fill:false,pointRadius:0,borderDash:[6,4]});
      }
      return new Chart(ctx,{
        type:'line',
        data:{labels:[],datasets:datasets},
        options:{
          responsive:true,
          maintainAspectRatio:false,
          animation:false,
          scales:{
            x:{display:true,grid:{display:false}},
            y:{display:true,min:minY,max:maxY,grid:{color:'#e0e0e0'}}
          },
          plugins:{legend:{display:true,position:'top'}}
        }
      });
    }

    window.onload=async function(){
      console.log('Initializing charts...');
      fcCharts.dry=makeChart(document.getElementById('fc_dry').getContext('2d'),'건조로 온도 2존 예측',thresholds.dry,97,101);
      fcCharts.quench=makeChart(document.getElementById('fc_quench').getContext('2d'),'소입로 온도 1존 예측',thresholds.quench,840,860);
      fcCharts.furnace=makeChart(document.getElementById('fc_furnace').getContext('2d'),'소입로 온도 4존 예측',thresholds.furnace,850,870);

      rtCharts.dry=makeChart(document.getElementById('rt_dry').getContext('2d'),'건조로 온도 2존',null,97,101);
      rtCharts.quench=makeChart(document.getElementById('rt_quench').getContext('2d'),'소입로 온도 1존',null,840,860);
      rtCharts.furnace=makeChart(document.getElementById('rt_furnace').getContext('2d'),'소입로 온도 4존',null,850,870);
      console.log('Charts initialized');

      async function tick(){
        try {
          const r=await fetch('/latest');
          const d=await r.json();
          
          const t=(d.timestamp||'').split(' ')[1];
          const s=d.sections;
          
          // 실시간 차트 업데이트
          rtCharts.dry.data.labels.push(t);
          rtCharts.dry.data.datasets[0].data.push(s.dry_temp.values[1]);
          rtCharts.quench.data.labels.push(t);
          rtCharts.quench.data.datasets[0].data.push(s.furnace_temp.values[0]);
          rtCharts.furnace.data.labels.push(t);
          rtCharts.furnace.data.datasets[0].data.push(s.furnace_temp.values[3]);
          
          Object.values(rtCharts).forEach(c=>{
            if(c.data.labels.length>60){
              c.data.labels.shift();
              c.data.datasets[0].data.shift();
            }
            c.update('none');
          });
          
          // 예측 차트 업데이트
          const f=d.forecast;
          
          if(f && f.labels && f.labels.length > 0 && f.series){
            ['dry','quench','furnace'].forEach(k=>{
              const ck=fcCharts[k];
              const keymap={dry:'dry_temp_2',quench:'quench_temp_1',furnace:'quench_temp_4'};
              const key=keymap[k];
              
              if(f.series[key] && f.series[key].length > 0){
                ck.data.labels=f.labels.map(x=>x.split(' ')[1]);
                ck.data.datasets[0].data=f.series[key];
                
                if(ck.data.datasets[1]){
                  ck.data.datasets[1].data=Array(f.labels.length).fill(thresholds[k]);
                }
                
                ck.update('none');
              }
            });
            
            // 램프 업데이트
            const lamps={
              dry:document.getElementById('lamp_dry'),
              quench:document.getElementById('lamp_quench'),
              furnace:document.getElementById('lamp_furnace')
            };
            const a=f.alarms;
            Object.entries(a).forEach(([k,v])=>{
              const n=k.includes('dry')?'dry':k.includes('temp_1')?'quench':'furnace';
              lamps[n].classList.remove('green','red');
              lamps[n].classList.add(v?'red':'green');
            });
          }
          
          document.getElementById('status').textContent=t+' | '+d.current_index+'/'+d.total_rows+' | 예측 기준: '+(f.last_from||'N/A');
        } catch(error) {
          console.error('Error in tick:', error);
        }
      }
      
      tick();
      setInterval(tick,1000);
    };
  </script>
</body>
</html>
"""

with open("templates/main.html", "w", encoding="utf-8") as f:
    f.write(main_html)

# ---- dashboard.html (상세 공정 페이지 - 전체 공정 실시간 모니터링) ----
dashboard_html = r"""<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>상세 공정 페이지</title>
  <script src="https://cdn.jsdelivr.net/npm/chart.js@4.4.0/dist/chart.umd.min.js"></script>
  <style>
    body { 
      font-family: 'Malgun Gothic', Arial; 
      margin: 0; 
      padding: 16px; 
      background-color: #f5f7fa; 
    }
    h1 { 
      text-align: center; 
      color: #2c3e50; 
      margin: 10px 0 20px; 
      font-size: 26px; 
    }
    .btn-container {
      text-align: center;
      margin-bottom: 20px;
    }
    .btn { 
      display: inline-block;
      padding: 10px 20px; 
      border: none; 
      border-radius: 6px;
      background: #28a745; 
      color: #fff; 
      font-size: 14px; 
      cursor: pointer;
      text-decoration: none;
    }
    .btn:hover { 
      background: #218838; 
    }
    
    .section-grid {
      display: grid;
      grid-template-columns: repeat(auto-fit, minmax(450px, 1fr));
      gap: 20px;
      max-width: 1400px;
      margin: 0 auto;
    }
    
    .section-card {
      background: #fff;
      border-radius: 8px;
      padding: 16px;
      box-shadow: 0 2px 6px rgba(0,0,0,0.08);
    }
    
    .section-card h2 {
      margin: 0 0 12px 0;
      font-size: 18px;
      color: #34495e;
      border-bottom: 2px solid #3498db;
      padding-bottom: 8px;
    }
    
    canvas {
      width: 100% !important;
      height: 220px !important;
    }
    
    .status-bar {
      text-align: center;
      font-size: 13px;
      color: #555;
      margin-top: 20px;
      padding: 10px;
      background: #fff;
      border-radius: 6px;
      box-shadow: 0 1px 3px rgba(0,0,0,0.06);
    }
  </style>
</head>
<body>
  <h1>상세 공정 모니터링</h1>
  
  <div class="btn-container">
    <a href="/" class="btn">메인 대시보드로 돌아가기</a>
  </div>
  
  <div class="section-grid">
    <div class="section-card">
      <h2>건조 1존 OP</h2>
      <canvas id="chart_dry_op_1"></canvas>
    </div>
    
    <div class="section-card">
      <h2>건조 2존 OP</h2>
      <canvas id="chart_dry_op_2"></canvas>
    </div>
    
    <div class="section-card">
      <h2>건조로 온도 1 Zone</h2>
      <canvas id="chart_dry_temp_1"></canvas>
    </div>
    
    <div class="section-card">
      <h2>건조로 온도 2 Zone</h2>
      <canvas id="chart_dry_temp_2"></canvas>
    </div>
    
    <div class="section-card">
      <h2>세정기</h2>
      <canvas id="chart_wash"></canvas>
    </div>
    
    <div class="section-card">
      <h2>소입 1존 OP</h2>
      <canvas id="chart_quench_1"></canvas>
    </div>
    
    <div class="section-card">
      <h2>소입 2존 OP</h2>
      <canvas id="chart_quench_2"></canvas>
    </div>
    
    <div class="section-card">
      <h2>소입 3존 OP</h2>
      <canvas id="chart_quench_3"></canvas>
    </div>
    
    <div class="section-card">
      <h2>소입 4존 OP</h2>
      <canvas id="chart_quench_4"></canvas>
    </div>
    
    <div class="section-card">
      <h2>소입로 CP 값</h2>
      <canvas id="chart_furnace_op"></canvas>
    </div>
    
    <div class="section-card">
      <h2>소입로 온도 1 Zone</h2>
      <canvas id="chart_furnace_temp_1"></canvas>
    </div>
    
    <div class="section-card">
      <h2>소입로 온도 2 Zone</h2>
      <canvas id="chart_furnace_temp_2"></canvas>
    </div>
    
    <div class="section-card">
      <h2>소입로 온도 3 Zone</h2>
      <canvas id="chart_furnace_temp_3"></canvas>
    </div>
    
    <div class="section-card">
      <h2>소입로 온도 4 Zone</h2>
      <canvas id="chart_furnace_temp_4"></canvas>
    </div>
  </div>
  
  <div class="status-bar" id="status">
    데이터 수신 중...
  </div>

  <script>
    const COLORS = ["#3498db", "#e67e22", "#2ecc71", "#9b59b6", "#e74c3c", "#1abc9c"];
    const charts = {};
    const MAX_POINTS = 60;

    function createChart(canvasId, label, color) {
      const ctx = document.getElementById(canvasId).getContext('2d');
      return new Chart(ctx, {
        type: 'line',
        data: {
          labels: [],
          datasets: [{
            label: label,
            data: [],
            borderColor: color,
            backgroundColor: color + '20',
            borderWidth: 2,
            fill: true,
            tension: 0.3,
            pointRadius: 0
          }]
        },
        options: {
          responsive: true,
          maintainAspectRatio: false,
          animation: false,
          scales: {
            x: {
              display: true,
              grid: { display: false }
            },
            y: {
              display: true,
              grid: { color: '#e0e0e0' }
            }
          },
          plugins: {
            legend: { display: false }
          }
        }
      });
    }

    window.onload = function() {
      charts.dry_op_1 = createChart('chart_dry_op_1', '건조 1존 OP', COLORS[0]);
      charts.dry_op_2 = createChart('chart_dry_op_2', '건조 2존 OP', COLORS[1]);
      charts.dry_temp_1 = createChart('chart_dry_temp_1', '건조로 온도 1 Zone', COLORS[2]);
      charts.dry_temp_2 = createChart('chart_dry_temp_2', '건조로 온도 2 Zone', COLORS[3]);
      charts.wash = createChart('chart_wash', '세정기', COLORS[4]);
      charts.quench_1 = createChart('chart_quench_1', '소입 1존 OP', COLORS[5]);
      charts.quench_2 = createChart('chart_quench_2', '소입 2존 OP', COLORS[0]);
      charts.quench_3 = createChart('chart_quench_3', '소입 3존 OP', COLORS[1]);
      charts.quench_4 = createChart('chart_quench_4', '소입 4존 OP', COLORS[2]);
      charts.furnace_op = createChart('chart_furnace_op', '소입로 CP 값', COLORS[3]);
      charts.furnace_temp_1 = createChart('chart_furnace_temp_1', '소입로 온도 1 Zone', COLORS[4]);
      charts.furnace_temp_2 = createChart('chart_furnace_temp_2', '소입로 온도 2 Zone', COLORS[5]);
      charts.furnace_temp_3 = createChart('chart_furnace_temp_3', '소입로 온도 3 Zone', COLORS[0]);
      charts.furnace_temp_4 = createChart('chart_furnace_temp_4', '소입로 온도 4 Zone', COLORS[1]);

      async function updateData() {
        try {
          const response = await fetch('/latest');
          const data = await response.json();
          
          const timeLabel = (data.timestamp || '').split(' ')[1] || '';
          const sections = data.sections;

          updateChart(charts.dry_op_1, timeLabel, sections.dry_op.values[0]);
          updateChart(charts.dry_op_2, timeLabel, sections.dry_op.values[1]);
          updateChart(charts.dry_temp_1, timeLabel, sections.dry_temp.values[0]);
          updateChart(charts.dry_temp_2, timeLabel, sections.dry_temp.values[1]);
          updateChart(charts.wash, timeLabel, sections.wash.values[0]);
          updateChart(charts.quench_1, timeLabel, sections.quench.values[0]);
          updateChart(charts.quench_2, timeLabel, sections.quench.values[1]);
          updateChart(charts.quench_3, timeLabel, sections.quench.values[2]);
          updateChart(charts.quench_4, timeLabel, sections.quench.values[3]);
          updateChart(charts.furnace_op, timeLabel, sections.furnace_op.values[0]);
          updateChart(charts.furnace_temp_1, timeLabel, sections.furnace_temp.values[0]);
          updateChart(charts.furnace_temp_2, timeLabel, sections.furnace_temp.values[1]);
          updateChart(charts.furnace_temp_3, timeLabel, sections.furnace_temp.values[2]);
          updateChart(charts.furnace_temp_4, timeLabel, sections.furnace_temp.values[3]);

          document.getElementById('status').textContent = 
            `${timeLabel} | 진행: ${data.current_index} / ${data.total_rows}`;

        } catch (error) {
          console.error('데이터 업데이트 실패:', error);
        }
      }

      function updateChart(chart, label, value) {
        chart.data.labels.push(label);
        chart.data.datasets[0].data.push(value);

        if (chart.data.labels.length > MAX_POINTS) {
          chart.data.labels.shift();
          chart.data.datasets[0].data.shift();
        }

        chart.update('none');
      }

      updateData();
      setInterval(updateData, 1000);
    };
  </script>
</body>
</html>
"""

with open("templates/dashboard.html", "w", encoding="utf-8") as f:
    f.write(dashboard_html)

# =========================================================
#  실행
# =========================================================
if __name__ == "__main__":
    print("\n========================================")
    print("실시간 공정 대시보드 실행 중")
    print("접속 주소: http://localhost:5432")
    print("========================================\n")
    app.run(host="0.0.0.0", port=5432, debug=False, threaded=True)

StandardScaler loaded: 18 features
warm-up 데이터: 300 rows (2022-01-03 11:27:10 ~ 2022-01-03 11:32:09)
Flask 실시간 데이터: 6894 rows (2022-01-03 11:32:10 ~ 2022-01-03 13:27:03)

모델 로딩 시작...
  - 건조로 온도 2 Zone: 로드 성공
  - 소입로 온도 1 Zone: 로드 성공
  - 소입로 온도 4 Zone: 로드 성공

초기 warm-up 예측 시작...
warm-up 데이터 준비 완료: 300행

예측 함수 호출 - 입력 데이터 크기: 300
리샘플링 후 데이터 크기: 30
예측 시간 라벨 생성 완료: 60개

[건조로 온도 2 Zone] 예측 시작
  원본 데이터 추출: 30개, 범위: 99.23~100.10
  정규화 완료: mean=100.01, scale=0.35, dtype=float32
  입력 shape: (1, 30, 1), dtype: float32
  모델 추론 완료: 60개 예측값
  역정규화 완료: 범위 100.15~100.82
  알람 상태: False (임계값: 98.88)

[소입로 온도 1 Zone] 예측 시작
  원본 데이터 추출: 30개, 범위: 858.80~860.32
  정규화 완료: mean=859.20, scale=3.56, dtype=float32
  입력 shape: (1, 30, 1), dtype: float32
  모델 추론 완료: 60개 예측값
  역정규화 완료: 범위 859.42~860.33
  알람 상태: False (임계값: 850.13)

[소입로 온도 4 Zone] 예측 시작
  원본 데이터 추출: 30개, 범위: 859.90~860.21
  정규화 완료: mean=860.01, scale=0.51, dtype=float32
  입력 shape: (1, 30, 1), dtype: float32
  모델 추론 완료: 60개 예측값
  역정규화 완료: 범위 859.9

/Users/ijh/Desktop/보고서/venv/lib/python3.10/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5432
 * Running on http://172.30.1.61:5432
Press CTRL+C to quit
127.0.0.1 - - [03/Nov/2025 13:52:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2025 13:52:15] "GET /latest HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2025 13:52:16] "GET /latest HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2025 13:52:17] "GET /latest HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2025 13:52:18] "GET /latest HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2025 13:52:19] "GET /latest HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2025 13:52:20] "GET /latest HTTP/1.1" 200 